<a href="https://colab.research.google.com/github/movie112/INU-DILAB/blob/main/INU-DILAB/dacon/dacon101/DACON_101_lv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DACON_101: 머신러닝이 처음이라면 🤔   
🏃‍♀️ [데이터 다운로드] 부터 ️[대회 제출] 까지 🏃‍♂
## Lv2. 결측치 보간법과 랜덤포레스트로 따릉이 데이터 예측하기
----
### - 전처리 - 
#### 전처리 1/2 결측치 대체 평균 (fillna({mean}))
- 원하는 피쳐의 결측치를 해당 피쳐의 평균값으로 대체


In [ ]:
df.fillna({'칼럼명':int(df['칼럼명'].mean)}, inplace=True)

#### 전처리 2/2 결측치 대체 보간(interpolate())
- 피쳐의 정보성을 강조하기 위해 보간법을 사용해 결측치 채움
- Lv2에서 다루는 따릉이데이터의 경우 피쳐들은 기상정보들이며 데이터의 순서는 시간 순서
- 따라서 결측치들을 이전 행(직전시간)과 다음 행(직후시간)의 평균으로 보간하는 것은 상당히 합리적
- Python pandas 의 interpolate() method 를 사용

In [ ]:
df.interpolate(inplace=True)

---
### - 모델링 - 
#### 모델링 1/2 랜덤포레스트 개념, 선언(RandomForestRegressor())
- 랜덤포레스트: 여러 개의 의사결정나무를 만들어서 이들의 평균으로 예측의 성능을 높이는 방법, 앙상블(Ensemble) 기법
- 주어진 하나의 데이터로부터 여러 개의 랜덤 데이터셋을 추출해, 각 데이터셋을 통해 모델을 여러 개 만들기 가능

In [1]:
# 랜덤포레스트회귀 모델을 선언
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

#### 모델링 2/2 랜덤포레스트를 평가척도에 맞게 학습(criterion='mse')
- 랜덤포레스트 모듈의 옵션 중 criterion을 통해 어떤 평가척도를 기준으로 훈련할 것인지 정하기 가능
- 따릉이 대회의 평가지표는 RMSE 입니다. RMSE 는 MSE 평가지표의 루트를 씌운 것으로서, 모델을 선언할 때 criterion = ‘mse’ 옵션으로 구현
<img src="https://lh5.googleusercontent.com/Yaq0wt7syFCLanNde8_xvMdFZ9BCe-rxXjeAOrGbctKtSr9XICo4Llf6iOmbz5JnHcst7UhiWlexWPhGEH5WDf1u_5SZb2TuxMYLL-CuqLJrNTCJ3ovbcs27udsElDmNJ596RfLl" width="300px" height="100px"></img>

In [ ]:
model = RandomForestRegressor(criterion = 'mse')

---
### - 튜닝 -
#### 튜닝 1/5 랜덤포레스트 변수중요도 확인(feature_importances_)
- fit()으로 모델 학습 후 feature_importances_ 속성(attribute) 으로 변수의 중요도 파악
- 변수의 중요도: 예측변수를 결정할 때 각 피쳐가 얼마나 중요한 역할을 하는지에 대한 척도, 낮다면 해당 피쳐를 제거하는 것이 모델의 성능 up

In [ ]:
model.feature_importances_

#### 튜닝 2/5 변수 제거(drop())
- 변수중요도가 낮은 피쳐를 파악하고 나면 차례대로 하나씩 피쳐를 제거하면서 모델을 새로 훈련   
1️⃣ 의미가 없는 feature인 id 와 count를 drop한 X_train_1 훈련df을 새로 생성, 예측을 할 때 test는 훈련 셋과 동일한 피쳐를 가져야 해서 동일하게 피쳐를 drop한 test_1 df 생성   
2️⃣ hour_bef_windspeed와 hour_bef_pm2.5피쳐도 추가로 drop을 수행하면서 위 과정 반복, 그럼 총 3쌍의 X_train셋과 test셋 생성   
3️⃣ 각 모델로 예측한 예측값들을 submission 에 저장한 후, 리더보드에 제출해 점수를 비교해보세요.   

In [ ]:
# X_train에서 drop할 피쳐의 경우의 수 대로 3개의 X_train 을 생성
X_train_1 = train.drop(['drop 할 피쳐'], axis=1)

# 각 train에 따라 동일하게 피쳐를 drop한 test셋들을 생성
test_1 = test.drop(['drop 할 피쳐'], axis = 1)

# 각 X_train에 대해 모델 훈련
model_input_var1 = RandomForestRegressor(criterion = 'mse')
model_input_var1.fit(X_train_1, Y_train)

# 각 모델로 test셋들을 예해주세요.
y_pred_1 = model_input_var1.predict(test_1)

# 각 결과들을 submission 파일로 저장
submission_1 = pd.read_csv('data/submission.csv')
submission_1['count'] = y_pred_1
submission_1.to_csv('sub_1.csv', index = False)

#### 튜닝 3/5 하이퍼파라미터 / GridSearch 개념(정지규칙)
- 정지규칙
> 1. 최대깊이(max_depth): 최대로 내려갈 수 있는 depth. 뿌리노드로부터 내려갈 수 있는 깊이를 지정하며 작을수록 트리는 작아짐.
> 2. 최소 노드크기(min_samples_split): 노드를 분할하기 위한 데이터 수. 해당 노드에 이 값보다 적은 확률변수 수가 있다면 stop. 작을수록 트리는 커짐.
> 3. 최소 향상도(min_impurity_decrease): 노드를 분할하기 위한 최소 향상도. 향상도가 설정값 이하라면 더 분할X, 작을수록 트리는 커짐.
> 4. 비용 복잡도(Cost-complexity): 트리가 커지는 것에 대해 패널티 계수를 설정해서 불순도와 트리가 커지는 것에 대해 복잡도를 계산

- 하이퍼파라미터 튜닝: 정지규칙 값들 설정, 정지규칙들을 종합적으로 고려해 최적의 조건값을 설정
- 다양한 방법론 중 Best 성능을 나타내는 GridSearch는 완전 탐색(Exhaustive Search) 을 사용
    - 가능한 모든 조합 중에서 가장 우수한 조합 탐색. 완전 탐색이어서 오래 걸린다는 단점

#### 튜닝 4/5 GridSearch 구현(GridSearchCV())
- GridSearchCV 모듈로 완전탐색 하이퍼파라미터 튜닝을 구현

In [ ]:
from sklearn.model_selection import GridSearchCV

model = RandomForestRegressor(criterion = 'mse', random_state=2020)

params = {'n_estimators': [200, 300, 500],
          'max_features': [5, 6, 8],
          'min_samples_leaf': [1, 3, 5]}

greedy_CV = GridSearchCV(model, param_grid=params, cv = 3, n_jobs = -1)
greedy_CV.fit(X_train, Y_train)